In [21]:
import sys

class FakeROB:

    def __init__(self, instruction_list_string):
        self.ZERO_REPLACEMENT = 564654
        self.fake_rob_queue = []
        self.currIndex = 0
        data_string = instruction_list_string.split("\n")
        index = 1
        for instruction in data_string:
            
            temp_ins_list = instruction.split(" ")
            if len(temp_ins_list) > 1:
                dst_temp = int(temp_ins_list[2])
                src1_temp = int(temp_ins_list[3])
                src2_temp = int(temp_ins_list[4])
                
                if dst_temp == 0:
                    dst_temp = self.ZERO_REPLACEMENT
            
                if src1_temp == 0:
                    src1_temp = self.ZERO_REPLACEMENT
                    
                if src2_temp == 0:
                    src2_temp = self.ZERO_REPLACEMENT
                
                self.fake_rob_queue.append({
                    "index":index,
                    "tag":temp_ins_list[0],
                    "operation_type": int(temp_ins_list[1]),
                    "dst": dst_temp,
                    "src1": src1_temp,
                    "src2": src2_temp,
                    "IF_cycle":-1,
                    "IF_duration":-1,
                    "ID_cycle":-1,
                    "ID_duration":-1,
                    "IS_cycle":-1,
                    "IS_duration":-1,
                    "EX_cycle":-1,
                    "EX_duration":-1,
                    "WB_cycle":-1,
                    "WB_duration":-1,
                    "current_state":None,
                    "ready": False,
                    "execution_timer":-1
                })
                index = index + 1
                
        self.instruction_count = len(self.fake_rob_queue)
    
    def Pop(self):
        
        if self.currIndex + 1 < self.instruction_count:
            index = self.currIndex
            self.currIndex = self.currIndex + 1
            return self.fake_rob_queue[index]
        else:
            return None
    
        
class Sim:
    IF = "IF"
    ID = "ID"
    IS = "IS"
    EX = "EX"
    WB = "WB"
    
    EXECUTE_CYCLE_LATENCY_DICT = {0:1, 1:2, 2:5}
    
    def __init__(self, data,scheduling_queue_size, peak_fetch_dispatch_issue_rate):
        self.currentCycle = 0
        self.fakeRob = FakeROB(data)
        self.scheduling_queue_size = scheduling_queue_size
        self.peak_fetch_dispatch_issue_rate = peak_fetch_dispatch_issue_rate
        self.fetch_list = []
        self.dispatch_list = []
        self.issue_list = []
        self.execute_list = []
        self.writeback_list = []
        self.register_state = {}
        self.reservation_station = {}
     
    
    def PrintList(self, state_List):
        for state in state_List:
            print(state)
    
    def GetFormattedOutput(self):
        for i in range(0, len(simulator.fakeRob.fake_rob_queue) - 1):
            data = simulator.fakeRob.fake_rob_queue[i]
            dst_temp = int(data["dst"])
            src1_temp = int(data["src1"])
            src2_temp = int(data["src2"])
                
            if dst_temp == simulator.fakeRob.ZERO_REPLACEMENT:
                dst_temp = 0
            
            if src1_temp == simulator.fakeRob.ZERO_REPLACEMENT:
                src1_temp = 0
                    
            if src2_temp == simulator.fakeRob.ZERO_REPLACEMENT:
                src2_temp = 0
            
            
            print( str(data["index"] - 1) + 
                  " fu{" + str(data["operation_type"]) 
                  + "} src{" + str(src1_temp) + "," + str(src2_temp) 
                  + "} dst{" + str(dst_temp) 
                  + "} IF{" +  str(data["IF_cycle"]) + "," + str(data["IF_duration"]) 
                  + "} ID{" + str(data["ID_cycle"]) + "," + str(data["ID_duration"])  
                  + "} IS{" + str(data["IS_cycle"]) + "," + str(data["IS_duration"]) 
                  + "} EX{" + str(data["EX_cycle"]) + "," + str(data["EX_duration"])  
                  + "} WB{" + str(data["WB_cycle"]) + "," + str(data["WB_duration"])  + "}")
    
        instruction_count = len(simulator.fakeRob.fake_rob_queue) - 1
        cycle_count = simulator.currentCycle
        print("number of instructions = {}".format(instruction_count))
        print("number of cycles       = {}".format(cycle_count))
        print("IPC                    = {:0.5f}".format(instruction_count/cycle_count))
        
    def ValidateFile(self, txtFile):
            file = open(txtFile, "r")
            lines = file.readlines()
            index = 0
            for i in range(0, len(simulator.fakeRob.fake_rob_queue) - 1):
                data = simulator.fakeRob.fake_rob_queue[i]
                dst_temp = int(data["dst"])
                src1_temp = int(data["src1"])
                src2_temp = int(data["src2"])

                if dst_temp == simulator.fakeRob.ZERO_REPLACEMENT:
                    dst_temp = 0

                if src1_temp == simulator.fakeRob.ZERO_REPLACEMENT:
                    src1_temp = 0

                if src2_temp == simulator.fakeRob.ZERO_REPLACEMENT:
                    src2_temp = 0
                
                
                our_line = (str(data["index"] - 1) + 
                  " fu{" + str(data["operation_type"]) 
                  + "} src{" + str(src1_temp) + "," + str(src2_temp) 
                  + "} dst{" + str(dst_temp) 
                  + "} IF{" +  str(data["IF_cycle"]) + "," + str(data["IF_duration"]) 
                  + "} ID{" + str(data["ID_cycle"]) + "," + str(data["ID_duration"])  
                  + "} IS{" + str(data["IS_cycle"]) + "," + str(data["IS_duration"]) 
                  + "} EX{" + str(data["EX_cycle"]) + "," + str(data["EX_duration"])  
                  + "} WB{" + str(data["WB_cycle"]) + "," + str(data["WB_duration"])  + "}")
                
                if(our_line != lines[index].strip()):
                    print("Ours")
                    print(our_line)
                    print("Expected")
                    print(lines[index].strip())
                    break
                index = index + 1
    
    def IsReservationStationInstructionReady(self, instruction):
        reservation_item = self.reservation_station[instruction["index"]]
        
        if reservation_item["QJ"] == -1 and reservation_item["QK"] == -1:
            return True
        else:
   
            valid = True
            
            if(reservation_item["QJ"] != -1):
                value = self.reservation_station.get(reservation_item["QJ"])
                if(value != None):
                    valid = False
                #else:
                #    print("Cycle: " + str(self.currentCycle) + " Index was not updated: " + str(reservation_item["QJ"]) + " in instruction: " + str(instruction["index"]))
                    
            if(reservation_item["QK"] != -1):
                value = self.reservation_station.get(reservation_item["QK"])
                if(value != None):
                    valid = False
                #else:
                #    print("Cycle: " + str(self.currentCycle) + " Index was not updated: " + str(reservation_item["QK"]) + " in instruction: " + str(instruction["index"]))
            
            return valid
       
    
    def DeleteFromReservationStation(self, instruction):
        del self.reservation_station[instruction["index"]]
        
        #registerValue = self.CheckRegisterState(instruction["dst"])
        #if registerValue != False and registerValue == instruction["index"]:
        #    self.UpdateRegisterState(instruction["index"], False)
        
    def RemoveInstructionReservationStation(self, instruction):
        indexToCleanup = instruction["index"]
        
        self.DeleteFromReservationStation(instruction)
        
        for key in self.reservation_station.keys():
            if(self.reservation_station[key]["QJ"] == indexToCleanup):
                self.reservation_station[key]["QJ"] = -1
                
            if(self.reservation_station[key]["QK"] == indexToCleanup):
                self.reservation_station[key]["QK"] = -1
        
    def AddInstructionReservationStation(self, instruction):
        
        #if(instruction["index"] == 3):
        #    print("Cycle: " + str(self.currentCycle) + " Adding Index: " + str(instruction["index"]))
        
        value = self.reservation_station.get(instruction["index"])
        VJ = -1
        VK = -1
        QJ = -1
        QK = -1
        

        
        if value == None:
            if(self.CheckRegisterState(instruction["src1"]) != False):
                QJ = self.CheckRegisterState(instruction["src1"])
            #else:
                #print("Value of " + str(self.CheckRegisterState(instruction["src1"])))
            
            if(self.CheckRegisterState(instruction["src2"]) != False):
                QK = self.CheckRegisterState(instruction["src2"])
            #else:
                #print("Value of " + str(self.CheckRegisterState(instruction["src2"])))
        
            self.UpdateRegisterState(instruction["dst"], instruction["index"])
            
            
            self.reservation_station[instruction["index"]] = {
                                                           "OP": instruction["operation_type"],
                                                           "VJ":VJ,
                                                           "VK":VK,
                                                           "QJ":QJ,
                                                           "QK":QK
                                                            }
            
    def UpdateRegisterState(self, register, value):
        if register == -1:
            return
        else:
            self.register_state[register] = value
            
    def CheckRegisterState(self, register):
        if register == -1:
            return False
        
        value = self.register_state.get(register)
        
        if value == None:
            return False
        else:
            return value
        
    def FakeRetire(self):
        for writeback_instruction in self.writeback_list:
            if(writeback_instruction["WB_cycle"] == -1):
                writeback_instruction["WB_cycle"] = self.currentCycle
        
            writeback_instruction["current_state"] = Sim.WB
        
        while len(self.writeback_list) != 0:
            writeback_instruction = self.writeback_list.pop()
            writeback_instruction["WB_duration"] = 1
            writeback_instruction["WB_cycle"] = self.currentCycle
            
            if(writeback_instruction["index"] == self.CheckRegisterState(writeback_instruction["dst"])):
                self.UpdateRegisterState(writeback_instruction["dst"], False)

            
        return

    def Execute(self):
        # Execute instructions
        for execute_instruction in self.execute_list:
            if(execute_instruction["EX_cycle"] == -1):
                execute_instruction["EX_cycle"] = self.currentCycle
        
            execute_instruction["current_state"] = Sim.EX
            execute_instruction["ready"] = False
        
        
        ready_list = []
        for execute_instruction in self.execute_list:
            
            #if execute_instruction["EX_cycle"] + Sim.EXECUTE_CYCLE_LATENCY_DICT[execute_instruction["operation_type"]] < self.currentCycle + 2:
            if execute_instruction["execution_timer"] == self.currentCycle:
                #print("Execution Finished at {} in cycle {}".format(str(execute_instruction["index"]), str(self.currentCycle)))
                execute_instruction["ready"] = True
                execute_instruction["EX_duration"] = Sim.EXECUTE_CYCLE_LATENCY_DICT[execute_instruction["operation_type"]]
                
                ready_list.append(execute_instruction)
                
        for ready_instruction in ready_list:

            self.RemoveInstructionReservationStation(ready_instruction)
            self.writeback_list.append(ready_instruction)
            self.execute_list.remove(ready_instruction)
            
                    
        return

    def Issue(self):
        # Issue instructions
                                     
        #if(self.currentCycle == 8):
        #print("At cycle {} value of register state 14 is {}".format(str(self.currentCycle),str(self.CheckRegisterState(14))))
        #print(self.fakeRob.fake_rob_queue[4])
        
        for issue_instruction in self.issue_list:
            
            if(issue_instruction["IS_cycle"] == -1):
                issue_instruction["IS_cycle"] = self.currentCycle
                issue_instruction["current_state"] = Sim.IS
                
        temp_list = sorted(self.issue_list, key=lambda d: d["index"]) 
        self.issue_list = temp_list
        
        for issue_instruction in self.issue_list:
            self.AddInstructionReservationStation(issue_instruction)
        
        ready_list = []
        execution_cap = self.peak_fetch_dispatch_issue_rate + 1 

        for issue_instruction in self.issue_list:
            
            value = self.reservation_station.get(issue_instruction["index"])
            if(execution_cap > 0 and value != None and self.IsReservationStationInstructionReady(issue_instruction) == True):
                ready_list.append(issue_instruction)
                execution_cap = execution_cap - 1
        
        for ready_instruction in ready_list:
            ready_instruction["execution_timer"] = self.currentCycle + Sim.EXECUTE_CYCLE_LATENCY_DICT[ready_instruction["operation_type"]]
            ready_instruction["IS_duration"] = self.currentCycle + 1 - ready_instruction["IS_cycle"]

            self.execute_list.append(ready_instruction)
            self.issue_list.remove(ready_instruction)      
        
        return
    
    def DebugPrintList(self, listToPrint):
        print([i["index"] for i in listToPrint])

    def Dispatch(self):
        # Dispatch instructions
        to_remove = []
        for dispatch_instruction in self.dispatch_list:
            
            if(dispatch_instruction["ID_cycle"] == -1):
                dispatch_instruction["ID_cycle"] = self.currentCycle
                dispatch_instruction["current_state"] = Sim.ID 

                
            if(dispatch_instruction["current_state"] == Sim.IS):
                to_remove.append(dispatch_instruction)
        
        for rm_instruction in to_remove:
            self.dispatch_list.remove(rm_instruction)
        
        ready_list = []
        issue_cap = 0
       
        issue_cap =  self.scheduling_queue_size - len(self.issue_list)
        
        for dispatch_instruction in self.dispatch_list:
            
            if issue_cap == 0:
                break
            
            ready_list.append(dispatch_instruction)
            
            issue_cap = issue_cap - 1
            
        for ready_instruction in ready_list:
            ready_instruction["ID_duration"] = self.currentCycle + 1 - ready_instruction["ID_cycle"]
            self.issue_list.append(ready_instruction)
            #self.dispatch_list.remove(ready_instruction)
            
        return

    def Fetch(self):
        # Fetch instructions
        while len(self.fetch_list) < self.peak_fetch_dispatch_issue_rate:
            instruction_fetched = self.fakeRob.Pop()
            if instruction_fetched == None:
                break
            else:
                self.fetch_list.append(instruction_fetched)
                
        maxSendCount = min(self.peak_fetch_dispatch_issue_rate,  (self.peak_fetch_dispatch_issue_rate) * 2 - len(self.dispatch_list)) 
        
        for i in range(0, maxSendCount):
            if len(self.fetch_list) > 0:
                instruction_fetched = self.fetch_list[0]
                self.dispatch_list.append(instruction_fetched)
                instruction_fetched["IF_cycle"] = self.currentCycle
                instruction_fetched["current_state"] = Sim.IF
                instruction_fetched["IF_duration"] = 1
                self.fetch_list.remove(instruction_fetched)
                

                
        return

    def Advance_Cycle(self):
        
        debug_cycle = 10
        
        if self.currentCycle == debug_cycle:
            print("Current Cycle: " + str(self.currentCycle))
            #print("Register State")
            #print(self.register_state)
            print("Execution Count: " + str(len(self.execute_list)))
            print("Reservation Count: " + str(len(self.reservation_station.keys())))
            print("Issue Count: " + str(len(self.issue_list)))
            print("Dispatch Count: " + str(len(self.dispatch_list)))
            print("fetch Count: " + str(len(self.fetch_list)))
            #print("Schedule Window Count: " + str(len(self.re)))
            #self.PrintList(self.issue_list)
           
           # self.PrintList(self.execute_list)
           
        
        if(self.currentCycle == debug_cycle):
            print("____Reservation Debug___")
            print("Current Cycle: " + str(self.currentCycle))
            print("Reservation Station")
            print(self.reservation_station)
            print("Register State")
            for i in self.register_state.keys():
                print("register {} : {}".format(i, self.register_state[i] - 1))
            #print(self.register_state)
            print("____Reservation Debug End___")
            
            for issue_instruction in self.issue_list:
                if (issue_instruction["index"] == 1136):
                    print("Instruction is ready: " + str(self.IsReservationStationInstructionReady(issue_instruction)))
                           
        self.currentCycle = self.currentCycle + 1
        
        #self.DebugPrintList(self.dispatch_list)
        

        if len(self.fetch_list) == 0 and len(self.dispatch_list) == 0 and len(self.issue_list) == 0 and len(self.execute_list) == 0 and len(self.writeback_list) == 0:
            return False
        else:
            return True


    def Main(self):
        while True:
    
            self.FakeRetire()
            self.Execute()
            self.Issue()
            self.Dispatch()
            self.Fetch()

            if(self.Advance_Cycle() == False):
                #print("End Of Sim")
                break

In [22]:
def ReadFile(txtFile):
    file = open(txtFile, "r")
    lines = file.readlines()
    data_string = ""
    for line in lines:
        data_string += line
        
    return data_string + "FFFFF -1 -1 -1 -1\n"

In [23]:
data = ReadFile("val_trace_gcc.txt")
simulator = Sim(data,8,8)

In [24]:
simulator.Main()
simulator.ValidateFile("pipe_8_8_gcc.txt")

Current Cycle: 10
Execution Count: 8
Reservation Count: 10
Issue Count: 8
Dispatch Count: 16
fetch Count: 3
____Reservation Debug___
Current Cycle: 10
Reservation Station
{5: {'OP': 2, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': -1}, 6: {'OP': 2, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': -1}, 7: {'OP': 0, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': 5}, 19: {'OP': 0, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': -1}, 23: {'OP': 0, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': -1}, 24: {'OP': 1, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': -1}, 25: {'OP': 0, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': -1}, 26: {'OP': 0, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': 25}, 27: {'OP': 0, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': -1}, 28: {'OP': 1, 'VJ': -1, 'VK': -1, 'QJ': -1, 'QK': 22}}
Register State
register 14 : 4
register 15 : 24
register 19 : 26
register 16 : 5
register 3 : 21
register 23 : -1
register 18 : -1
register 2 : 20
register 17 : 23
register 4 : 27
____Reservation Debug End___


In [25]:
simulator.GetFormattedOutput()

0 fu{0} src{29,14} dst{-1} IF{0,1} ID{1,1} IS{2,1} EX{3,1} WB{4,1}
1 fu{2} src{29,-1} dst{14} IF{0,1} ID{1,1} IS{2,1} EX{3,5} WB{8,1}
2 fu{2} src{29,-1} dst{15} IF{0,1} ID{1,1} IS{2,1} EX{3,5} WB{8,1}
3 fu{2} src{30,-1} dst{19} IF{0,1} ID{1,1} IS{2,1} EX{3,5} WB{8,1}
4 fu{2} src{14,-1} dst{14} IF{0,1} ID{1,1} IS{2,6} EX{8,5} WB{13,1}
5 fu{2} src{15,-1} dst{16} IF{0,1} ID{1,1} IS{2,6} EX{8,5} WB{13,1}
6 fu{0} src{29,14} dst{-1} IF{0,1} ID{1,1} IS{2,11} EX{13,1} WB{14,1}
7 fu{0} src{19,-1} dst{3} IF{0,1} ID{1,1} IS{2,6} EX{8,1} WB{9,1}
8 fu{1} src{0,0} dst{23} IF{1,1} ID{2,1} IS{3,1} EX{4,2} WB{6,1}
9 fu{1} src{0,0} dst{18} IF{1,1} ID{2,1} IS{3,1} EX{4,2} WB{6,1}
10 fu{0} src{3,0} dst{-1} IF{1,1} ID{2,1} IS{3,6} EX{9,1} WB{10,1}
11 fu{0} src{0,-1} dst{2} IF{1,1} ID{2,1} IS{3,1} EX{4,1} WB{5,1}
12 fu{0} src{3,2} dst{-1} IF{1,1} ID{2,2} IS{4,5} EX{9,1} WB{10,1}
13 fu{0} src{19,-1} dst{3} IF{1,1} ID{2,2} IS{4,4} EX{8,1} WB{9,1}
14 fu{0} src{3,-1} dst{2} IF{1,1} ID{2,2} IS{4,5} EX{9,1} WB{10